In [1]:
from datasets import load_dataset
from datasets import load_from_disk
from transformers import AutoTokenizer
import torch
import os
import json
from pathlib import Path

# dataset = load_dataset("ashaba1in/small_openwebtext")
# dataset.save_to_disk('data/datasets/small_openwebtext')

/home/boris/micromamba/envs/nlp_hw3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 1000000/1000000 [00:35<00:00, 28202.61 examples/s]


In [13]:
dataset = load_from_disk('data/datasets/small_openwebtext')
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-v0.1')

In [17]:
def _tokenize_function(sample):
    return tokenizer(sample['text'])

In [43]:
subset = dataset['train'].select(range(10000))

subset_tok = subset.map(_tokenize_function, batched=True)

In [44]:
subset_tok['input_ids'][0][:5] # массив из токенов

[1, 4194, 28733, 581, 28733]

In [46]:
counter = 0
absolut_i = 0
for tok_text in subset_tok['input_ids']:
    i = 0
    while (start := i * 256) + 256 <= len(tok_text):
        torch.save(tok_text[start: start + 256], f'data/datasets/extrasmall_openwebtext_tok/train/text_{absolut_i}.pt')
        i += 1
        absolut_i += 1
    
    assert len(tok_text) - start < 256

    torch.save(tok_text[start: ], f'data/datasets/extrasmall_openwebtext_tok/train/text_{absolut_i}.pt')
    absolut_i += 1


In [47]:
index = []
part = 'train'
data_dir = Path('data/datasets/extrasmall_openwebtext_tok')
for text_path in os.listdir(data_dir / part):
    index.append(
        {
            "text_path": str(data_dir / part / text_path)
        }
    )

index_path = Path(data_dir / 'train_index.json')

with index_path.open("w") as f:
    json.dump(index, f, indent=2)

In [48]:
index[1]

{'text_path': 'data/datasets/extrasmall_openwebtext_tok/train/text_34325.pt'}

In [1]:
len(index)

NameError: name 'index' is not defined